# Exercise

The ACME inc. tool supply company manages its operations with 3 csv files:

1. `customers.csv` keeps customer information:
    * `id` is a numeric customer id
    * `firstname` is the customer's first name
    * `lastname` is the customer's last name
2. `products.csv` keeps product info:
    * `id` is a numeric product id
    * `name` is the human-readable name
    * `cost` is the product cost in euros
3. `orders.csv` keeps order information:
    * `id` is a numeric order id
    * `customer` is the numeric id of the customer who created the order
    * `products` is a space-separated list of product ids ordered by the customer

Manually dealing with those files is hard and error-prone, and they've asked for your help writing some code to make their lives easier.

### Task 1

Right now the `orders.csv` doesn't have total order cost information.

We need to use the data in these files to emit a `order_prices.csv` file with the following columns:
* `id` the numeric id of the order
* `euros` the total cost of the order

In [1]:
import pandas as pd

orders_df = pd.read_csv("orders.csv", usecols=['id', 'customer', 'products'])
products_df = pd.read_csv("products.csv", usecols=['id', 'name', 'cost'])

#Showing the first 5 columns of the dataframe
print(orders_df.head())
print(products_df.head())

   id  customer           products
0   0         0            1 0 1 0
1   1        22  0 5 0 4 5 3 2 1 1
2   2        57              5 2 4
3   3        20          4 3 0 2 4
4   4        28        5 0 5 0 3 4
   id         name      cost
0   0  screwdriver  2.981164
1   1       wrench  6.490396
2   2       hammer  2.903732
3   3       sickle  8.901570
4   4       candle  9.806495


In [2]:
list_total = []

for index, row in orders_df.iterrows():
    #selecting purchased elements
    elements = row['products'].split(" ")
    #computing total cost of all the products
    total = sum([products_df.loc[products_df["id"]==int(i), "cost"].iloc[0] for i in elements])
    list_total.append(total)
    

In [3]:
# Create new dataframe with only the 2 columns required
order_prices = pd.DataFrame(data=list(zip(orders_df['id'], list_total)), columns=['order_id', 'total_price'])

print(order_prices.head())

   order_id  total_price
0         0    18.943120
1         1    61.425421
2         2    23.145479
3         3    34.399455
4         4    45.540896


In [4]:
#Saving it into a csv
order_prices.to_csv("order_prices.csv", columns=['order_id', 'total_price'], index=False)

### Task 2

The marketing department wants to know which customers are interested in each product; they've asked for a `product_customers.csv` file that, for each product, gives the list of customers who have purchased this product:
* `id` numeric product id
* `customer_ids` a space-separated list of customer ids of the customers who have purchased this product

In [5]:
#Creating dictionary to efficiently store the data
dict_customers = {}
for index, row in orders_df.iterrows():
    #selecting purchased elements
    elements = row['products'].split(" ")
    
    for element in elements:
        #if the product is new to the dictionary
        if element not in dict_customers:
            dict_customers[element] = [row['customer']]
        #if it already exist just append new customer
        else:
            if row['customer'] not in dict_customers[element]:
                dict_customers[element].append(row['customer'])

#sort dictionary by order id
dict_customers = {k : dict_customers[k] for k in sorted(dict_customers)}

In [6]:
dict_customers

{'0': [0,
  22,
  20,
  28,
  40,
  32,
  5,
  45,
  37,
  38,
  6,
  44,
  50,
  24,
  54,
  59,
  15,
  21,
  34,
  19,
  47,
  48,
  46,
  10,
  17,
  29],
 '1': [0,
  22,
  40,
  32,
  45,
  38,
  51,
  6,
  44,
  34,
  3,
  50,
  24,
  15,
  5,
  41,
  47,
  46,
  35,
  29,
  10,
  17,
  9,
  58],
 '2': [22,
  57,
  20,
  40,
  5,
  45,
  37,
  51,
  6,
  44,
  54,
  8,
  15,
  21,
  41,
  48,
  46,
  29,
  24,
  17,
  9],
 '3': [22,
  20,
  28,
  32,
  5,
  38,
  51,
  34,
  50,
  24,
  54,
  21,
  41,
  47,
  46,
  44,
  29,
  17,
  9],
 '4': [22,
  57,
  20,
  28,
  51,
  24,
  59,
  36,
  8,
  15,
  21,
  5,
  34,
  19,
  41,
  44,
  32,
  35,
  10,
  17,
  9],
 '5': [22,
  57,
  28,
  32,
  5,
  37,
  38,
  6,
  44,
  34,
  3,
  50,
  24,
  54,
  59,
  15,
  25,
  21,
  45,
  19,
  41,
  47,
  10,
  17,
  9,
  29]}

In [7]:
#save dictionary to csv
product_customers_df = pd.DataFrame(data=list(zip(dict_customers.keys(), dict_customers.values())), columns=['product_id', 'customers'])
print(product_customers_df)
product_customers_df.to_csv("product_customers.csv", columns=['product_id', 'customers'], index=False)

  product_id                                          customers
0          0  [0, 22, 20, 28, 40, 32, 5, 45, 37, 38, 6, 44, ...
1          1  [0, 22, 40, 32, 45, 38, 51, 6, 44, 34, 3, 50, ...
2          2  [22, 57, 20, 40, 5, 45, 37, 51, 6, 44, 54, 8, ...
3          3  [22, 20, 28, 32, 5, 38, 51, 34, 50, 24, 54, 21...
4          4  [22, 57, 20, 28, 51, 24, 59, 36, 8, 15, 21, 5,...
5          5  [22, 57, 28, 32, 5, 37, 38, 6, 44, 34, 3, 50, ...


### Task 3

To evaluate our customers, we need a `customer_ranking.csv` containing the following columns, ranked in descending order by total_euros:
* `id` numeric id of the customer
* `firstname` customer first name
* `lastname` customer last name
* `total_euros` total euros this customer has spent on products

In [8]:
#Loading the previously created order_prices dataframe
order_prices_df = pd.read_csv("order_prices.csv", usecols=['order_id', 'total_price'])
orders_df = pd.read_csv("orders.csv", usecols=['id', 'customer', 'products'])
customers_df = pd.read_csv("customers.csv", usecols=['id', 'firstname', 'lastname'])

In [9]:
#Creating dictionary to efficiently store the data
dict_customers_rankings = {}
for index, row in order_prices_df.iterrows():
    
    #Getting the customer who made the order
    customer_id = orders_df.loc[orders_df['id'] == row['order_id'], 'customer'].iloc[0]
    
    #Creating dictionary inside the original one to store multiple entries
    if customer_id not in dict_customers_rankings:
            dict_customers_rankings[customer_id] = {'total': row['total_price'], 
                                                    'firstname': customers_df.loc[customers_df['id'] == customer_id, 'firstname'].iloc[0],
                                                    'lastname': customers_df.loc[customers_df['id'] == customer_id, 'lastname'].iloc[0]
                                                   }
    else:
        #If already exists just update total cost
        dict_customers_rankings[customer_id]['total'] += row['total_price']
    

In [10]:
dict_customers_rankings

{0: {'total': 18.94312018282366, 'firstname': 'John', 'lastname': 'Maxwell'},
 22: {'total': 124.71069029464272,
  'firstname': 'Paul',
  'lastname': 'Lavoisier'},
 57: {'total': 23.1454792209952, 'firstname': 'Jane', 'lastname': 'Feynman'},
 20: {'total': 34.39945536782433, 'firstname': 'Paul', 'lastname': 'Fermi'},
 28: {'total': 45.54089635777856,
  'firstname': 'Ringo',
  'lastname': 'Lavoisier'},
 40: {'total': 15.356455867079756,
  'firstname': 'Sam',
  'lastname': 'Lavoisier'},
 32: {'total': 119.16389695712095, 'firstname': 'Samuel', 'lastname': 'Fermi'},
 5: {'total': 99.14373610351268, 'firstname': 'John', 'lastname': 'Pasteur'},
 45: {'total': 35.79133727218077, 'firstname': 'Mary', 'lastname': 'Feynman'},
 37: {'total': 37.190652332205616,
  'firstname': 'Sam',
  'lastname': 'Heisenberg'},
 38: {'total': 65.07085261235663, 'firstname': 'Sam', 'lastname': 'Fermi'},
 51: {'total': 34.592589673186325,
  'firstname': 'Sarah',
  'lastname': 'Feynman'},
 6: {'total': 22.810544398

In [12]:
#SORTING PART

#First we obtain a list with the customers ids sorted by total amount spent
customers_sorted_list = sorted(dict_customers_rankings.keys(), key=lambda x: (dict_customers_rankings[x]['total']), reverse=True)
print(customers_sorted_list)

#From that list, we created a new one with all the data stored by customer
ranking = []
for item in customers_sorted_list:
    ranking.append([item, dict_customers_rankings[item]['firstname'], dict_customers_rankings[item]['lastname'], dict_customers_rankings[item]['total']])


[34, 44, 24, 22, 32, 5, 10, 29, 38, 41, 21, 15, 59, 50, 17, 28, 3, 9, 47, 37, 46, 45, 51, 20, 19, 54, 57, 6, 25, 0, 35, 40, 8, 36, 58, 48]


In [13]:
ranking

[[34, 'Samuel', 'Lavoisier', 145.2543359308485],
 [44, 'Mary', 'Fermi', 138.4106269396071],
 [24, 'Ringo', 'Maxwell', 136.82767039246306],
 [22, 'Paul', 'Lavoisier', 124.71069029464272],
 [32, 'Samuel', 'Fermi', 119.16389695712095],
 [5, 'John', 'Pasteur', 99.14373610351268],
 [10, 'James', 'Lavoisier', 73.1779650450406],
 [29, 'Ringo', 'Pasteur', 70.0317519380194],
 [38, 'Sam', 'Fermi', 65.07085261235663],
 [41, 'Sam', 'Pasteur', 60.77799949217107],
 [21, 'Paul', 'Feynman', 54.64120246756335],
 [15, 'Peter', 'Feynman', 54.27075731112488],
 [59, 'Jane', 'Pasteur', 52.64239549683009],
 [50, 'Sarah', 'Fermi', 51.16828884378742],
 [17, 'Peter', 'Pasteur', 50.91273763436682],
 [28, 'Ringo', 'Lavoisier', 45.54089635777856],
 [3, 'John', 'Feynman', 40.3416936820254],
 [9, 'James', 'Feynman', 38.53744542169881],
 [47, 'Mary', 'Pasteur', 38.279942132039736],
 [37, 'Sam', 'Heisenberg', 37.190652332205616],
 [46, 'Mary', 'Lavoisier', 36.668828177075135],
 [45, 'Mary', 'Feynman', 35.7913372721807

In [14]:
#save dictionary to csv
customer_ranking_df = pd.DataFrame(data=ranking, columns=['customer_id', 'firstname', 'lastname', 'total_price'])
print(customer_ranking_df)
customer_ranking_df.to_csv("customer_ranking.csv", columns=['customer_id', 'firstname', 'lastname', 'total_price'], index=False)

    customer_id firstname    lastname  total_price
0            34    Samuel   Lavoisier   145.254336
1            44      Mary       Fermi   138.410627
2            24     Ringo     Maxwell   136.827670
3            22      Paul   Lavoisier   124.710690
4            32    Samuel       Fermi   119.163897
5             5      John     Pasteur    99.143736
6            10     James   Lavoisier    73.177965
7            29     Ringo     Pasteur    70.031752
8            38       Sam       Fermi    65.070853
9            41       Sam     Pasteur    60.777999
10           21      Paul     Feynman    54.641202
11           15     Peter     Feynman    54.270757
12           59      Jane     Pasteur    52.642395
13           50     Sarah       Fermi    51.168289
14           17     Peter     Pasteur    50.912738
15           28     Ringo   Lavoisier    45.540896
16            3      John     Feynman    40.341694
17            9     James     Feynman    38.537445
18           47      Mary     P